<a href="https://colab.research.google.com/github/junaiskv94/Creditcard_xml_project/blob/main/credit_card_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# Mounting google drieve
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
# import os moule
import os
from xml.etree import cElementTree as ET
# Get the list of all files and directories
file_names=os.listdir('/content/drive/MyDrive/Coding_data')

In [64]:
# Defining empty list for each variable
person=[]
DPD_percentage=[]
Total_disbursed_amount=[]
DPD_max_number_months=[]


In [65]:
# removing the unnecessary last element
file_names.pop(-1)

'.ipynb_checkpoints'

In [66]:
# Checking the files
file_names

['customer898231_loan899591_crif_report.html.xml',
 'customer1195586_loan1197471_crif_report.html.xml',
 'customer773504_loan774538_crif_report.html.xml',
 'customer14235_loan14235_crif_report.html.xml',
 'customer1129550_loan1131339_crif_report.html.xml',
 'customer1113697_loan1115483_crif_report.html.xml',
 'customer787561_loan788638_crif_report.html.xml',
 'customer40409_loan40409_crif_report.html.xml',
 'customer16475_loan16475_crif_report.html.xml',
 'customer794397_loan795497_crif_report.html.xml']

In [67]:
for index,name in enumerate(file_names): # Iterating over entire file
  with open('/content/drive/MyDrive/Coding_data/'+name,'r') as f: # opening file
    data = f.read() # reading file
  root = ET.fromstring(data) # parses xml in to root element
  trades=[]
  disb_amount=[]
  for x in root.iter('LOAN-DETAILS'): # Iterate over the tree
    item=x.find('COMBINED-PAYMENT-HISTORY').text
    item='month:yyyy,000/XXX|month:yyyy,000/XXX|'
    trades.append(item)
    try: # correction for the error due to empty text
      amount=x.find('DISBURSED-AMT').text
    except:
      amount='0,0'
    disb_amount.append(amount)
    
  total_trade=len(trades)
  dpd_count=0
  for i in range(total_trade):# iterate over total trade
    try:
      payment_hist=trades[i].split('|')
      dpd=[h.split('/')[0][-3:] for h in payment_hist[:-1]] # extracting the DPD 
    except:
      dpd='000'
    for val in dpd:
      if val.isdigit():
        if int(val)>30: # checking for DPD more than 30 
          dpd_count+=1
          break;
  dpd_percentage=(dpd_count/total_trade)*100      
  # convert string to int 
  total_disb_amount=sum([int(val.replace(',','')) for val in disb_amount])
  dpd_max_month=[]
  for i in range(total_trade):
    dpd_value=0
    try:
      payment_hist=trades[i].split('|')
      dpd=[h.split('/')[0][-3:] for h in payment_hist[:-1]] 
    except:
       dpd='000' 
    for val in dpd:
      if val.isdigit():# checking whether a digit or not 
         if int(val)>30:
           dpd_value+=1
    dpd_max_month.append(dpd_value)
    max_month_due=max(dpd_max_month) # finding the max of month of dpd30+due
  # Appending the values in each iteration to the list
  person.append(index+1)
  DPD_percentage.append(dpd_percentage)
  Total_disbursed_amount.append(total_disb_amount)
  DPD_max_number_months.append(max_month_due)


In [68]:
# creating empty dictionary
dict={}
# Adding corresponding  key-values pairs
dict['Person']=person
dict['DPD_percentage']=DPD_percentage
dict['Total_disbursed_amount']=Total_disbursed_amount
dict['DPD_max_number_months']=DPD_max_number_months


In [56]:
# Importing panas 
import pandas as pd


In [59]:
# creating data frame
df=pd.DataFrame(dict)

In [60]:
df

,Person,DPD_percentage,Total_disbursed_amount,DPD_max_number_months
0,1,12.820513,12449012,18
1,2,41.176471,4471863,3
2,3,24.324324,4133365,36
3,4,18.918919,8532117,7
4,5,38.461538,795601,10
5,6,18.518519,1057320,4
6,7,16.666667,691839,2
7,8,17.857143,1075308,5
8,9,15.384615,2868467,5
9,10,14.285714,910791,6


In [61]:
# saving to csv file
df.to_csv('output.csv')